In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 3.5 MB 22.0 MB/s 
     |████████████████████████████████| 6.8 MB 49.2 MB/s 
     |████████████████████████████████| 1.2 MB 56.8 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 39.3 MB/s 
     |████████████████████████████████| 895 kB 39.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=6cb1ba0b4fec092b1767eece95b86250cf7d7c98a22b28b793127e4cfe0bfc09
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy import dot
from numpy.linalg import norm
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from google.colab import drive
drive.mount("/content/drive")
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

Mounted at /content/drive


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/931 [00:00<?, ?B/s]

In [3]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    # df_sorting = train_data.sort_values(by="score", ascending=False)
    # df_sorting = df_sorting.drop_duplicates(['A'])
    return train_data.loc[train_data['score'].idxmax()]['A']

#jhgan/ko-sroberta-multitask


In [5]:
import re
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

num = [1,2,3,4,5,6,7,8,9,10]
stopword = [ str(number)+"종" for number in num ] + ["혼합색상", "그린", "블루", "핑크", "레드", "블랙", "화이트", "그레이","옐로우", "크림", "차콜","노랑","실버","빨강","오렌지","베이지","퍼플","보라","옐로우","옐로우", "\n"]
stopword

def preprocessing (review, remove_stopwords = True ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()	

    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Zㄱ-힣0-9]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()
    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        stops = stopword
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        word = []
        stops = ["g", "ml", "kg", "개", "박스", "box", "p", "세트", "색상"]
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        for w in words:
            word.append(w)
            for s in stops:
                if (s in w):
                    word.remove(w)
                    break
        words = word
        

        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.	
        clean_review = ' '.join(words)

    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)
    if (len(clean_review) <= 5):
        clean_review = review

    return clean_review


# 40만개 10만개 분할


In [8]:
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/11bunga_data.csv')
train_data

clean_train = []
for review in train_data['11st_ItemNm']:
    clean_train.append(preprocessing(review, remove_stopwords = True))

train_data["name"] = clean_train
train_data.drop(train_data.columns[0],axis=1, inplace=True)
train_data.rename(columns={"name" : "Q", "naver_category" : "A"}, inplace=True)
train_data
#그냥 name to category
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)
train_data.to_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/preprocessing_11bunga.pkl')

# 여기 위에꺼 까지만 돌리면됨 아래는 돌릴 필요 없음 이거 파일에 번호 잘 보고 읽어와서 돌려 주세요.


In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/naver_c40_4.csv')
train_data.drop(train_data.columns[0],axis=1, inplace=True)
train_data.rename(columns={"name" : "Q", "middle_classification" : "A"}, inplace=True)
train_data

#그냥 name to category
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)
train_data.to_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/naver40_middle_4.pkl')

In [ ]:
kan_naver_cat = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/category to category.csv', encoding="cp949")
kan_naver_cat.columns = [0,"Q","A"]
kan_naver_cat

,0,Q,A
0,0.0,가공식품 조미료 종합조미료 천연/발효조미료,식품 가공식품
1,1.0,가공식품 조미료 종합조미료 식초,식품 가공식품
2,2.0,가공식품 조미료 종합조미료 천일염,식품 가공식품
3,3.0,가공식품 조미료 종합조미료 가공염,식품 가공식품
4,4.0,가공식품 조미료 종합조미료 설탕,식품 가공식품
...,...,...,...
2806,NaN,전문스포츠/레저 승마 승마용품 승마용품,스포츠/레저 기타스포츠용품
2807,NaN,전문스포츠/레저 암벽등반 암벽등반용품 암벽등반용품,스포츠/레저 기타스포츠용품
2808,NaN,전문스포츠/레저 기타레저용품 기타레저용품 기타레저용품,스포츠/레저 기타스포츠용품
2809,NaN,패션잡화 유아동패션잡화 유아동패션잡화 유아동신발,출산/육아 유아동의류


In [ ]:
# 칸 카테고리 to 네이버 중분류 카테고리
kan_naver_cat
kan_naver_cat['embedding'] = kan_naver_cat.apply(lambda row: model.encode(row.Q), axis = 1)
train_data = kan_naver_cat

In [ ]:
# 저장 로직
train_data.to_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/category_2_category.pkl')

# 40만개 네이버 데이터 

In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/data_naver_40.csv')
train_data.drop(train_data.columns[0],axis=1, inplace=True)
train_data.rename(columns={"name" : "Q", "middle_classification" : "A"}, inplace=True)
train_data

,classification,Q,brand,A
0,화장품/미용 베이스메이크업 컨실러,마루빌츠 케머플라지 크림컨실러+브러쉬or스펀지,maluwilzshop,화장품/미용 베이스메이크업
1,화장품/미용 베이스메이크업 컨실러,[한스킨] 블레미쉬 커버 12g 컨실러 트러블 홍조 다크 잡티커버,셀트리온스킨큐어 공식스토어,화장품/미용 베이스메이크업
2,화장품/미용 베이스메이크업 컨실러,포렌코즈 타투 워터프루프 스카 컨실러,forencos,화장품/미용 베이스메이크업
3,화장품/미용 베이스메이크업 컨실러,더샘 커버 퍼펙션 팁 컨실러 6.5g(SPF28),쇼핑몰별 최저가,화장품/미용 베이스메이크업
4,화장품/미용 베이스메이크업 컨실러,나스 래디언트 크리미 컨실러 6ml,쇼핑몰별 최저가,화장품/미용 베이스메이크업
...,...,...,...,...
426280,생활/건강 의료용품 석션기/네블라이저,영화의료기 HS-112 HS112 가정용 석션기 썩션기 전동식 의료용 흡인기 국내제조,유한메디칼,생활/건강 의료용품
426281,생활/건강 의료용품 석션기/네블라이저,산소 발생기 소형 수면 가정용 호흡기 노인 임산부 산,NaN,생활/건강 의료용품
426282,생활/건강 의료용품 석션기/네블라이저,대한 접은거즈 5CMX5CM 8P 200매 X10개 병원 의료용 9460492,NaN,생활/건강 의료용품
426283,생활/건강 의료용품 석션기/네블라이저,산소 발생기 소형 수면 가정용 호흡기 노인 임산부 산,NaN,생활/건강 의료용품


In [ ]:
#그냥 name to category
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)
train_data.to_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/naver40_middle.pkl')

#10만개 name to category


In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab_data/middle_classification.csv')

In [ ]:
train_data = train_data[["classification", "name", "middle_classification"]]
classification_list = list(train_data.classification)
middle_list = list(train_data.middle_classification)
abc_list = []
for x in range(len(middle_list)):
    split_list = classification_list[x].split()
    abc_list.append(split_list[0] + " " + middle_list[x])
train_data["A"] = abc_list
train_data.rename(columns = {"name" : "Q"}, inplace=True)
train_data
# def large_get(train_data_name):
#     list_name = list(train_data_name[2])
#     for x in range(len(list_name)):
#         split_list = list_name[x].split()
#         list_name[x] = split_list[0]+" "+split_list[1]
#     train_data_name["A"] = list_name
#     return train_data_name

# print(large_get(train_data))

,classification,Q,middle_classification,A
0,스포츠/레저 마라톤용품,나이키 스파크 라이트웨이트,마라톤용품,스포츠/레저 마라톤용품
1,스포츠/레저 마라톤용품,런닝 육상복 민소매 유니폼 마라톤복 세트,마라톤용품,스포츠/레저 마라톤용품
2,스포츠/레저 마라톤용품,플립벨트 FLIPBELT 런닝벨트 런닝밴드 러닝 스마트폰허리가방 마라톤힙색,마라톤용품,스포츠/레저 마라톤용품
3,스포츠/레저 마라톤용품,프리가든 러닝벨트 런닝 휴대폰 플립벨트 스마트폰 허리 밴드 벨크로 러닝백,마라톤용품,스포츠/레저 마라톤용품
4,스포츠/레저 마라톤용품,무지 벨크로 러닝벨트 플립벨트 밴드 스마트폰 허리 러닝용품,마라톤용품,스포츠/레저 마라톤용품
...,...,...,...,...
103173,출산/육아 유아동잡화 기타유아동잡화,몽베베 올리브빕,유아동잡화,출산/육아 유아동잡화
103174,출산/육아 유아동잡화 기타유아동잡화,빛나는구름 한복 소품 복주머니,유아동잡화,출산/육아 유아동잡화
103175,출산/육아 유아동잡화 기타유아동잡화,미소 패피선글라스,유아동잡화,출산/육아 유아동잡화
103176,출산/육아 유아동잡화 기타유아동잡화,장갑M 샌드박스 폴라오지 잠뜰,유아동잡화,출산/육아 유아동잡화


In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)
train_data.to_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/model/naver40_middle.pkl')

In [ ]:
return_similar_answer('홈플래닛 고속충전 케이블 5핀 5색 1.5m 5개')

,0,1,2,Q,3,A,embedding,score
96632,97293,97293,디지털/가전 휴대폰액세서리 휴대폰케이블,홈플래닛 고속충전케이블 C타입 2m 5개입,마켓 샤인,휴대폰액세서리,"[0.018336007, -0.29454577, 0.23593439, -0.0149...",0.877903
101345,102051,102051,디지털/가전 카메라/캠코더용품 DSLR카메라,데이터전송케이블 5M 라인 USB 30 충전 19121055W3435298005,NaN,카메라/캠코더용품,"[0.30573326, 0.21240644, 0.22813177, -0.538476...",0.735024
62205,62637,62637,생활/건강 공구 전기용품기타 전기용품,HIV전선 HFIX전선 전기선 케이블 1.5SQ 2.5SQ 4SQ,유일전선,공구,"[0.32377556, 0.0746244, -0.21690914, -0.688379...",0.728785
82440,83005,83005,디지털/가전 자동차기기 카오디오음향기기음향기기기타,AUX 억스 케이블 변환 젠더 2.5 3.5 파이,요즘 살림,자동차기기,"[0.81019163, 0.11892921, 0.2967295, -0.2750398...",0.726796
6374,6404,6404,스포츠/레저 오토바이/스쿠터 오토바이용품기타오토바이용품,배달대행 자석충전거치대 충전패드 교체형 젠더 C타입 5핀 아이폰 선택,Rational,오토바이/스쿠터,"[0.017559983, -0.25852582, 0.13348868, -0.0246...",0.707061
...,...,...,...,...,...,...,...,...
41944,42235,42235,출산/육아 아기간식 유아과자,베베쿡 사르르쿵 과자 치즈요거트 5개,NaN,아기간식,"[-0.17375039, 0.66791886, -0.44237652, 0.07141...",0.370074
64926,65371,65371,패션잡화 여성신발 실내화,사무실 거실 털실내화 털슬리퍼 스마일 양털 겨울실내화 5종,플랫28,여성신발,"[-0.019050252, 0.39361134, -0.081474856, -0.81...",0.370037
51044,51371,51371,출산/육아 수영복/용품 수경/수모/귀마개,아레나 ARENA 아동용 수경 5가지 AGW-380J AOAAG38,쇼핑몰별 최저가,수영복/용품,"[0.16741875, 0.06925999, -0.58089995, -0.31789...",0.369462
60855,61287,61287,스포츠/레저 스키/보드 스키/보드용품헬멧,아날로그플러스 크랭크 닉스 CRNK 스키헬멧 스노우보드헬멧 초경량 통풍,쇼핑몰별 최저가,스키/보드,"[-0.45638368, 0.32461715, 0.7144366, -0.072518...",0.314886


'휴대폰액세서리'

# 3000개 Kan category to naver middle category

In [ ]:
train_data = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/colab_data/category_2_category.pkl')

In [ ]:
data_category = train_data["A"].unique()

In [ ]:
train_data

,0,Q,A,embedding
0,0.0,가공식품 조미료 종합조미료 천연/발효조미료,식품 가공식품,"[0.24742524, 0.057393327, -0.41764966, -0.3797..."
1,1.0,가공식품 조미료 종합조미료 식초,식품 가공식품,"[0.31171533, 0.27334338, -0.49156782, -0.30727..."
2,2.0,가공식품 조미료 종합조미료 천일염,식품 가공식품,"[0.45889756, 0.32210755, 0.19666386, -0.658986..."
3,3.0,가공식품 조미료 종합조미료 가공염,식품 가공식품,"[0.14444481, 0.24607944, -0.17839701, -0.45316..."
4,4.0,가공식품 조미료 종합조미료 설탕,식품 가공식품,"[-0.10215004, 0.12851545, -0.08579867, -0.0188..."
...,...,...,...,...
2806,NaN,전문스포츠/레저 승마 승마용품 승마용품,스포츠/레저 기타스포츠용품,"[-0.053023744, 0.077895194, -0.3524724, 0.5073..."
2807,NaN,전문스포츠/레저 암벽등반 암벽등반용품 암벽등반용품,스포츠/레저 기타스포츠용품,"[0.08617684, -0.14468187, -0.2317923, 0.38944,..."
2808,NaN,전문스포츠/레저 기타레저용품 기타레저용품 기타레저용품,스포츠/레저 기타스포츠용품,"[-0.029529903, -0.1945483, -0.6072183, 0.09566..."
2809,NaN,패션잡화 유아동패션잡화 유아동패션잡화 유아동신발,출산/육아 유아동의류,"[0.40249443, 0.3238217, -0.9191139, 0.01274753..."


In [ ]:
return_similar_answer('여성패션 의류 원피스')

,0,Q,A,embedding,score
1517,1517.0,의류 패션의류 여성의류 원피스,패션의류 여성의류,"[0.3052323, -0.08632119, 0.052693717, -0.15443...",0.925016
2771,NaN,의류 언더웨어 여성언더웨어 여성내의,패션의류 여성언더웨어/잠옷,"[0.2611329, -0.14418314, -0.25347087, 0.153821...",0.774721
1535,1535.0,의류 패션의류 여성의류 임부복,패션의류 임부복,"[0.21361381, -0.24095066, -0.14370705, -0.1440...",0.730587
1824,1824.0,패션잡화 벨트 멜빵/여성벨트 여성벨트,패션잡화 벨트,"[0.08458839, -0.27128232, -0.08884247, 0.00734...",0.696516
1902,1902.0,패션잡화 패션소품 손수건 여성용,패션잡화 패션소품,"[0.5773512, -0.28984153, -0.28219795, -0.27799...",0.696015
...,...,...,...,...,...
1311,1311.0,디지털/가전 생활가전 헤어드라이어/고데기/매직기 매직기,디지털/가전 이미용가전,"[0.2861548, -0.1048832, -0.15875599, -0.467079...",0.159449
2503,NaN,교육/문화용품 서적 참고서/수험서 초등참고서,도서 참고서,"[0.8425348, 0.1437895, -0.34063488, -0.2709488...",0.136715
1644,1644.0,전문스포츠/레저 당구 당구용품 기타당구용품,스포츠/레저 당구용품,"[-0.58325404, -0.35179538, 0.07641343, -0.3431...",0.129849
2506,NaN,교육/문화용품 서적 참고서/수험서 수험서,도서 수험서,"[0.8734349, -0.12683514, -0.29935578, -0.44794...",0.122665


'패션의류 여성의류'

In [ ]:
print(data_name)
print(data_category)
count = 0
for x in range(len(data_category)):
    for y in range(len(data_name)):
        if (data_name[y] in data_category[x]):
            count += 1
            data_name[y] = data_category[x]
            print(data_category[x])
print(data_name)
print(count)

식품 가공식품
식품 아이스크림/빙수
식품 김치
출산/육아 분유
출산/육아 아기간식
출산/육아 이유식
식품 건강식품
식품 다이어트식품
식품 과자
식품 음료
식품 농산물
생활/건강 구강위생용품
출산/육아 물티슈
생활/건강 반려동물
생활/건강 자동차용품
생활/건강 주방용품
생활/건강 청소용품
생활/건강 세탁용품
생활/건강 생활용품
생활/건강 수집품
생활/건강 건강측정용품
생활/건강 정원/원예용품
생활/건강 공구
출산/육아 수유용품
패션잡화 모자
화장품/미용 남성화장품
화장품/미용 선케어
화장품/미용 마스크/팩
화장품/미용 클렌징
출산/육아 임산부용품
출산/육아 구강청결용품
생활/건강 의료용품
생활/건강 안마용품
생활/건강 냉온/찜질용품
생활/건강 좌욕/좌훈용품
생활/건강 문구/사무용품
스포츠/레저 골프
스포츠/레저 볼링
도서 잡지
생활/건강 악기
생활/건강 음반
디지털/가전 학습기기
디지털/가전 노트북
디지털/가전 모니터
디지털/가전 PC부품
디지털/가전 PC부품
디지털/가전 PC액세서리
디지털/가전 네트워크장비
디지털/가전 저장장치
디지털/가전 소프트웨어
디지털/가전 노트북액세서리
디지털/가전 노트북액세서리
디지털/가전 게임기/타이틀
디지털/가전 휴대폰
디지털/가전 태블릿PC
디지털/가전 휴대폰액세서리
디지털/가전 휴대폰액세서리
디지털/가전 영상가전
디지털/가전 음향가전
디지털/가전 멀티미디어장비
디지털/가전 주방가전
디지털/가전 생활가전
디지털/가전 이미용가전
디지털/가전 카메라/캠코더용품
디지털/가전 자동차기기
가구/인테리어 거실가구
가구/인테리어 수납가구
가구/인테리어 아동/주니어가구
가구/인테리어 인테리어소품
가구/인테리어 주방가구
가구/인테리어 침구단품
가구/인테리어 침구세트
가구/인테리어 침실가구
가구/인테리어 커튼/블라인드
패션의류 여성의류
패션의류 임부복
패션의류 남성의류
출산/육아 유아동의류
스포츠/레저 야구
스포츠/레저 배드민턴
스포츠/레저 축구
스포츠/레저 농구
스포츠/레저 배구
스포츠/레저 탁구
스포츠/레저 스쿼시
스포츠/레저 테니스
스포츠/

In [ ]:
for x in data_name:
    if (" " not in x):
        print(x)

#마라톤 기타스포츠